# Fine-tune Amazon Titan Text Express provided by Amazon Bedrock

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio. Also use ml.c5.2xlarge due to memory resources required*

In this notebook, we will fine-tune [Amazon Titan Text Lite](#https://docs.aws.amazon.com/bedrock/latest/userguide/titan-text-models.html) model provided by Amazon Bedrock for summarization use case.
You can choose from list of base models or fine-tune one of your previously fine tuned model.

## Prerequisites

 - Make sure you have executed `00_setup.ipynb` notebook.
 - Make sure you are using the same kernel and instance as `00_setup.ipynb` notebook.

<div class="alert alert-block alert-warning">
<b>Warning:</b> This notebook will create provisioned throughput for testing the fine-tuned model. Therefore, please make sure to delete the provisioned throughput as mentioned in the last section of the notebook, otherwise you will be charged for it, even if you are not using it.
</div>

In [1]:
!pip install -qU bert_score

In [2]:
# restart kernel for packages to take effect
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
## Fetching varialbes from `00_setup.ipynb` notebook. 
%store -r role_arn
%store -r s3_train_uri
%store -r s3_validation_uri
%store -r s3_test_uri
%store -r bucket_name

In [2]:
import pprint
pprint.pp(role_arn)
pprint.pp(s3_train_uri)
pprint.pp(s3_validation_uri)
pprint.pp(s3_test_uri)
pprint.pp(bucket_name)

'arn:aws:iam::183295442349:role/BedrockRole-1d8483cd-8e8d-42d4-9315-461a24d523f8'
's3://bedrock-customization-us-east-1-183295442349/fine-tuning-datasets/train/train-cnn-5K.jsonl'
's3://bedrock-customization-us-east-1-183295442349/fine-tuning-datasets/validation/validation-cnn-1K.jsonl'
's3://bedrock-customization-us-east-1-183295442349/fine-tuning-datasets/test/test-cnn-10.jsonl'
'bedrock-customization-us-east-1-183295442349'


## Setup

In [4]:
import warnings
warnings.filterwarnings('ignore')
import json
import os
import sys
import boto3
import time

In [5]:
session = boto3.session.Session()
region = session.region_name
sts_client = boto3.client('sts')
s3_client = boto3.client('s3')
aws_account_id = sts_client.get_caller_identity()["Account"]
bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

In [5]:
test_file_name = "test-cnn-10.jsonl"
data_folder = "fine-tuning-datasets"

## Select the model you would like to fine-tune
You will have to provide the `base_model_id` for the model you are planning to fine-tune. You can get that using `list_foundation_models` API as follows: 
```
for model in bedrock.list_foundation_models(
    byCustomizationType="FINE_TUNING")["modelSummaries"]:
    for key, value in model.items():
        print(key, ":", value)
    print("-----\n")
```

In [8]:
for model in bedrock.list_foundation_models(
    byCustomizationType="FINE_TUNING")["modelSummaries"]:
    for key, value in model.items():
        print(key, ":", value)
    print("-----\n")

modelArn : arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0
modelId : amazon.titan-image-generator-v1:0
modelName : Titan Image Generator G1
providerName : Amazon
inputModalities : ['TEXT', 'IMAGE']
outputModalities : ['IMAGE']
customizationsSupported : ['FINE_TUNING']
inferenceTypesSupported : ['PROVISIONED']
modelLifecycle : {'status': 'ACTIVE'}
-----

modelArn : arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v2:0
modelId : amazon.titan-image-generator-v2:0
modelName : Titan Image Generator G1 v2
providerName : Amazon
inputModalities : ['TEXT', 'IMAGE']
outputModalities : ['IMAGE']
customizationsSupported : ['FINE_TUNING']
inferenceTypesSupported : ['PROVISIONED', 'ON_DEMAND']
modelLifecycle : {'status': 'ACTIVE'}
-----

modelArn : arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0:300k
modelId : amazon.nova-pro-v1:0:300k
modelName : Nova Pro
providerName : Amazon
inputModalities : ['TEXT', 'IMAGE', 'VIDEO']
outputM

In [9]:
base_model_id = "amazon.titan-text-lite-v1:0:4k"

Next you will need to provide the `customization_job_name`, `custom_model_name` and `customization_role` which will be used to create the fine-tuning job. 

In [11]:
from datetime import datetime
ts = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

customization_job_name = f"model-finetune-job-{ts}"
custom_model_name = f"finetuned-model-{ts}"
customization_role = role_arn

## Create fine-tuning job
<div class="alert alert-block alert-info">
<b>Note:</b> Fine-tuning job will take around 1hr to complete.</div>

Amazon Titan text model customization hyperparameters: 
- `epochs`: The number of iterations through the entire training dataset and can take up any integer values in the range of 1-10, with a default value of 5.
- `batchSize`: The number of samples processed before updating model parameters and can take up any integer values in the range of 1-64, with a default value of 1.
- `learningRate`:	The rate at which model parameters are updated after each batch	which can take up a float value betweek 0.0-1.0 with a default value set to	1.00E-5.
- `learningRateWarmupSteps`: The number of iterations over which the learning rate is gradually increased to the specified rate and can take any integer value between 0-250 with a default value of 5.

For guidelines on setting hyper-parameters refer to the guidelines provided [here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-guidelines.html)

In [12]:
# Select the customization type from "FINE_TUNING" or "CONTINUED_PRE_TRAINING". 
customization_type = "FINE_TUNING"

In [13]:
# Define the hyperparameters for fine-tuning Titan text model
hyper_parameters = {
        "epochCount": "2",
        "batchSize": "1",
        "learningRate": "0.00003",
    }


s3_bucket_config=f's3://{bucket_name}/outputs/output-{custom_model_name}'
# Specify your data path for training, validation(optional) and output
training_data_config = {"s3Uri": s3_train_uri}

validation_data_config = {
        "validators": [{
            # "name": "validation",
            "s3Uri": s3_validation_uri
        }]
    }

output_data_config = {"s3Uri": s3_bucket_config}


# Create the customization job
training_job_response = bedrock.create_model_customization_job(
    customizationType=customization_type,
    jobName=customization_job_name,
    customModelName=custom_model_name,
    roleArn=customization_role,
    baseModelIdentifier=base_model_id,
    hyperParameters=hyper_parameters,
    trainingDataConfig=training_data_config,
    validationDataConfig=validation_data_config,
    outputDataConfig=output_data_config
)
training_job_response

{'ResponseMetadata': {'RequestId': 'c52839c0-d205-47df-92b5-1ec630172b15',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Sat, 26 Apr 2025 10:11:52 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c52839c0-d205-47df-92b5-1ec630172b15'},
  'RetryAttempts': 0},
 'jobArn': 'arn:aws:bedrock:us-east-1:183295442349:model-customization-job/amazon.titan-text-lite-v1:0:4k/wdoaeno8w4iy'}

## Check fine-tuning job status

In [14]:
fine_tune_job = bedrock.get_model_customization_job(jobIdentifier=customization_job_name)["status"]
print(fine_tune_job)

while fine_tune_job == "InProgress":
    time.sleep(60)
    fine_tune_job = bedrock.get_model_customization_job(jobIdentifier=customization_job_name)["status"]
    print (fine_tune_job)
    time.sleep(60)

InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
Completed


In [10]:
fine_tune_job = bedrock.get_model_customization_job(jobIdentifier=customization_job_name)

ValidationException: An error occurred (ValidationException) when calling the GetModelCustomizationJob operation: The provided job identifier is invalid.

In [9]:
pprint.pp(fine_tune_job)

NameError: name 'fine_tune_job' is not defined

In [ ]:
output_job_name = "model-customization-job-"+fine_tune_job['jobArn'].split('/')[-1]
output_job_name

Now we are ready to create [`provisioned throughput`](#) which is needed before you can do the inference on the fine-tuned model.

### Overview of Provisioned throughput
You specify Provisioned Throughput in Model Units (MU). A model unit delivers a specific throughput level for the specified model. The throughput level of a MU for a given Text model specifies the following:

- The total number of input tokens per minute – The number of input tokens that an MU can process across all requests within a span of one minute.

- The total number of output tokens per minute – The number of output tokens that an MU can generate across all requests within a span of one minute.

Model unit quotas depend on the level of commitment you specify for the Provisioned Throughput.

- For custom models with no commitment, a quota of one model unit is available for each Provisioned Throughput. You can create up to two Provisioned Throughputs per account.

- For base or custom models with commitment, there is a default quota of 0 model units. To request an increase, use the [limit increase form](https://support.console.aws.amazon.com/support/home#/case/create?issueType=service-limit-increase).

## Retrieve Custom Model
Once the customization job is finished, you can check your existing custom model(s) and retrieve the modelArn of your fine-tuned model.

In [14]:
# List your custom models
bedrock.list_custom_models()

{'ResponseMetadata': {'RequestId': '6fd8dd01-e7d6-4784-9d31-f04937ea7024',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 26 Apr 2025 14:39:27 GMT',
   'content-type': 'application/json',
   'content-length': '417',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6fd8dd01-e7d6-4784-9d31-f04937ea7024'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1:183295442349:custom-model/amazon.titan-text-lite-v1:0:4k/95e8y689gok7',
   'modelName': 'finetuned-model-2025-04-26-09-58-51',
   'creationTime': datetime.datetime(2025, 4, 26, 10, 11, 52, 38000, tzinfo=tzlocal()),
   'baseModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-text-lite-v1:0:4k',
   'baseModelName': '',
   'customizationType': 'FINE_TUNING',
   'ownerAccountId': '183295442349'}]}

In [7]:
model_id = bedrock.get_custom_model(modelIdentifier='finetuned-model-2025-04-26-09-58-51')['modelArn'] # custom_model_name of the previous session
model_id

'arn:aws:bedrock:us-east-1:183295442349:custom-model/amazon.titan-text-lite-v1:0:4k/95e8y689gok7'

## Create Provisioned Throughput
<div class="alert alert-block alert-info">
<b>Note:</b> Creating provisioned throughput will take around 20-30mins to complete.</div>

You will need to create provisioned throughput to be able to evaluate the model performance. You can do so through the [console].(https://docs.aws.amazon.com/bedrock/latest/userguide/prov-cap-console.html) or use the following api call:

In [8]:
import boto3 
boto3.client(service_name='bedrock')
provisioned_model_id = bedrock.create_provisioned_model_throughput(
 modelUnits=1,
 provisionedModelName='test-model', 
 modelId=model_id
)['provisionedModelArn']     

ValidationException: An error occurred (ValidationException) when calling the CreateProvisionedModelThroughput operation: The ProvisionedModelName test-model is already in use!

In [20]:
status_provisioning = bedrock.get_provisioned_model_throughput(provisionedModelId = provisioned_model_id)['status']

In [21]:
import time
while status_provisioning == 'Creating':
    time.sleep(60)
    status_provisioning = bedrock.get_provisioned_model_throughput(provisionedModelId=provisioned_model_id)['status']
    print(status_provisioning)
    time.sleep(60)

Creating
Creating
Creating
Creating
Creating
Creating
InService


## Invoke the Custom Model

Before invoking lets get the sample prompt from our test data. 

In [22]:
# Provide the prompt text 
test_file_path = f'{data_folder}/{test_file_name}'
with open(test_file_path) as f:
    lines = f.read().splitlines()

In [23]:
test_prompt = json.loads(lines[3])['prompt']
reference_summary = json.loads(lines[3])['completion']
pprint.pp(test_prompt)
print(reference_summary)

('Below is an instruction that describes a task, paired with an input that '
 'provides further context. Write a response that appropriately completes the '
 'request.\n'
 '\n'
 'instruction:\n'
 '\n'
 'Summarize the news article provided below.\n'
 '\n'
 'input:\n'
 '\n'
 'When Jessica Carey drove more than three hours to Washington to see her '
 "grandmother, the elderly lady didn't realise she was in store for two "
 'surprises. As Jessica walked through the door, she revealed she had had a '
 "portrait of a younger Patty Lawing tattooed on her forearm. Patty's "
 'heartwarming reaction was captured on film by a relative, showing her '
 'overcome with emotion at the artwork. Jessica surprised her grandmother '
 'Patty Lawing with the tattoo and it was captured on camera . Jessica had her '
 'husband Aaron Carey create the stunningly accurate tattoo of Patty on her '
 'forearm based on a childhood portrait. It showed her grandmother as a young '
 'woman, with long red hair, wearing a

In [24]:
prompt = f"""
{test_prompt}
"""

In [25]:
base_model_arn = f'arn:aws:bedrock:{region}::foundation-model/amazon.titan-text-lite-v1'

Make sure to construct model input following the format needed by titan text model following instructions [here](#https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-text.html). 
Please pay attention to the "Model invocation request body field" section in the `body` variable, which we will pass as payload to the custom model trained above. 

In [26]:
body = json.dumps(
    {
    "inputText": prompt,
    "textGenerationConfig": {
        "maxTokenCount": 2048,
        "stopSequences": ['User:'],
        "temperature": 0,
        "topP": 0.9
    }
    }
        )

accept = 'application/json'
contentType = 'application/json'

fine_tuned_response = bedrock_runtime.invoke_model(body=body, 
                                        modelId=provisioned_model_id, 
                                        accept=accept, 
                                        contentType=contentType)

base_model_response = bedrock_runtime.invoke_model(body=body, 
                                        modelId=base_model_arn, 
                                        accept=accept, 
                                        contentType=contentType)

fine_tuned_response_body = json.loads(fine_tuned_response.get('body').read())
base_model_response_body = json.loads(base_model_response.get('body').read())

print("Base model response: ", base_model_response_body["results"][0]["outputText"] + '\n')
print("Fine tuned model response:", fine_tuned_response_body["results"][0]["outputText"]+'\n')
print("Reference summary from test data: " , reference_summary)

Base model response:  Jessica Carey drove three hours to Washington to see her grandmother Patty Lawing .
She had a portrait of a younger Patty tattooed on her forearm .
Patty's reaction was captured on camera and it was priceless .

Fine tuned model response: Jessica Carey had her husband Aaron Carey tattoo a childhood portrait of her grandmother on her forearm .
Patty Lawing was so overwhelmed by the gesture that she began to shake and wave her arms .

Reference summary from test data:  response:

Jessica Carey had her grandmother's portrait tattooed on her forearm .
She then drove almost four hours to surprise her grandmother with it .
Her grandmother Patty Lawing's priceless reaction is captured on film .


## Evaluate the performance of the model 
In this section, we will use `BertScore` metrics  to evaluate the performance of the fine-tuned model as compared to base model to check if fine-tuning has improved the results.

- `BERTScore`: calculates the similarity between a summary and reference texts based on the outputs of BERT (Bidirectional Encoder Representations from Transformers), a powerful language model. [Medium article link](#https://haticeozbolat17.medium.com/bertscore-and-rouge-two-metrics-for-evaluating-text-summarization-systems-6337b1d98917)

In [27]:
base_model_generated_response = [base_model_response_body["results"][0]["outputText"]]
fine_tuned_generated_response = [fine_tuned_response_body["results"][0]["outputText"]]

In [28]:
from bert_score import score
reference_summary = [reference_summary]
fine_tuned_model_P, fine_tuned_R, fine_tuned_F1 = score(fine_tuned_generated_response, reference_summary, lang="en")
base_model_P, base_model_R, base_model_F1 = score(base_model_generated_response, reference_summary, lang="en")
print("F1 score: base model ", base_model_F1)
print("F1 score: fine-tuned model", fine_tuned_F1)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


F1 score: base model  tensor([0.9171])
F1 score: fine-tuned model tensor([0.8874])


## Conclusion
From the scores above and looking at the base model summary, fine-tuned model summary and reference summary, it clearly indicates that fine-tuning the model tends to improve the results on the task its trained on. We only used 1K records for training with 100 validation records and 2 epochs, and were able to get better results. 
You may want to adjust the learning_rate first, visualize training and validation metrics to understand the performance of training job, before increase the size of your data. 

<div class="alert alert-block alert-info">
<b>Tip:</b> 
    Please refer to the <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-guidelines.html" style="color: #3372FF">guidelines </a> provided for fine-tuning the model based on your task. </div>

## Delete provisioned througput
<div class="alert alert-block alert-warning">
<b>Warning:</b> Please make sure to delete providsioned throughput as there will cost incurred if its left in running state, even if you are not using it. 
</div>

In [11]:
provisioned_model_id = 'arn:aws:bedrock:us-east-1:183295442349:provisioned-model/ltj10jy6wfpy'
bedrock.delete_provisioned_model_throughput(provisionedModelId=provisioned_model_id)

{'ResponseMetadata': {'RequestId': '9900acb6-881b-4066-a5f4-430587a47ba5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 28 Apr 2025 16:18:09 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9900acb6-881b-4066-a5f4-430587a47ba5'},
  'RetryAttempts': 0}}